
The general recipe is a short list of four main steps:

1.   Compose a function to **read** input data and prepare a Tensorflow Dataset;
2.   Define a **scoring** function that, given a (set of) query-document feature vector(s), produces a score indicating the query's level of relevance to the document;
3.   Create a **loss** function that measures how far off the produced scores from step (2) are from the ground truth; and,
4.   Define evaluation **metrics**.

## Imports

In [24]:
import tensorflow as tf
import tensorflow_ranking as tfr
import pandas as pd

tf.enable_eager_execution()
tf.executing_eagerly()

True

## Constants

In [297]:
# Store the paths to files containing training and test instances.
# As noted above, we will assume the data is in the LibSVM format
# and that the content of each file is sorted by query ID.

_TRAIN_DATA_PATH="../data/interim/df_train_test.libsvm"
_TEST_DATA_PATH="../data/interim/df_train_test.libsvm"

# Define a loss function. To find a complete list of available
# loss functions or to learn how to add your own custom function
# please refer to the tensorflow_ranking.losses module.
_LOSS="pairwise_logistic_loss"

# In the TF-Ranking framework, a training instance is represented
# by a Tensor that contains features from a list of documents
# associated with a single query. For simplicity, we fix the shape
# of these Tensors to a maximum list size and call it "list_size,"
# the maximum number of documents per query in the dataset.
# In this demo, we take the following approach:
#   * If a query has fewer documents, its Tensor will be padded
#     appropriately.
#   * If a query has more documents, we shuffle its list of
#     documents and trim the list down to the prescribed list_size.
_LIST_SIZE=100

# The total number of features per query-document pair.
# We set this number to the number of features in the MSLR-Web30K
# dataset.
_NUM_FEATURES=4

# Parameters to the scoring function.
_BATCH_SIZE=100
_HIDDEN_LAYER_DIMS=["20", "10"]

#_OUT_DIR = "../models/tfranking/"

## Function to read in data and form tensorflow dataset

Out train and test dataset is in the lib svm format which is normally used for Support Vector Machines. 

In [299]:
fo = open(_TRAIN_DATA_PATH)
i=0
for f in fo:
    if i != 10:
        print(f)
    else:
        break
    i+=1

0 qid:2558352 1:5 2:4694 3:4228 4:700

0 qid:2558352 1:3 2:5113 3:604 4:700

0 qid:2558352 1:6 2:4703 3:1420 4:700

1 qid:2558352 1:1 2:5380 3:1930 4:200

0 qid:2558353 1:4 2:6033 3:1099 4:2300

1 qid:2558353 1:2 2:4381 3:1612 4:300

0 qid:2558353 1:9 2:4382 3:1370 4:300

0 qid:2558353 1:5 2:4045 3:3532 4:700

0 qid:2558353 1:3 2:6033 3:799 4:700

0 qid:2558353 1:6 2:4078 3:1231 4:700



In this example they first number shows the target

1 = Important 

0 = Not important

qid: Describes which lines belong together

E.g. query 10 had 6 suggestions for plans and just one is important. Here we would take the suggested transport mode from this plan. 

### Input Pipeline

The first step to construct an input pipeline that reads your dataset and produces a `tensorflow.data.Dataset` object. In this example, we will invoke a LibSVM parser that is included in the `tensorflow_ranking.data` module to generate a `Dataset` from a given file.

We parameterize this function by a `path` argument so that the function can be used to read both training and test data files.

 Read tf DataSet
 
 Dic for feature

In [300]:
def input_fn(path):
    train_dataset = tf.data.Dataset.from_generator(
      tfr.data.libsvm_generator(path, _NUM_FEATURES, _LIST_SIZE),
      output_types=(
          {str(k): tf.float32 for k in range(1,_NUM_FEATURES+1)},
          tf.float32
      ),
      output_shapes=(
          {str(k): tf.TensorShape([_LIST_SIZE, 1])
            for k in range(1,_NUM_FEATURES+1)},
          tf.TensorShape([_LIST_SIZE])
      )
    )

    train_dataset = train_dataset.shuffle(1000).repeat().batch(_BATCH_SIZE)
    return train_dataset.make_one_shot_iterator().get_next()

In [301]:
train_dataset = tf.data.Dataset.from_generator(
      tfr.data.libsvm_generator(_TRAIN_DATA_PATH, _NUM_FEATURES, _LIST_SIZE),
      output_types=(
          {str(k): tf.float32 for k in range(1,_NUM_FEATURES+1)},
          tf.float32
      ),
      output_shapes=(
          {str(k): tf.TensorShape([_LIST_SIZE, 1])
            for k in range(1,_NUM_FEATURES+1)},
          tf.TensorShape([_LIST_SIZE])
      )
    )

train_dataset

<DatasetV1Adapter shapes: ({1: (100, 1), 2: (100, 1), 3: (100, 1), 4: (100, 1)}, (100,)), types: ({1: tf.float32, 2: tf.float32, 3: tf.float32, 4: tf.float32}, tf.float32)>

In [302]:
a=train_dataset.make_one_shot_iterator().get_next()

In [304]:
a[0]

{'1': <tf.Tensor 'IteratorGetNext_1:0' shape=(100, 1) dtype=float32>,
 '2': <tf.Tensor 'IteratorGetNext_1:1' shape=(100, 1) dtype=float32>,
 '3': <tf.Tensor 'IteratorGetNext_1:2' shape=(100, 1) dtype=float32>,
 '4': <tf.Tensor 'IteratorGetNext_1:3' shape=(100, 1) dtype=float32>}

## Estimator Creation


Next, we turn to the scoring function which is arguably at the heart of a TF Ranking model. The idea is to compute a relevance score for a (set of) query-document pair(s). The TF-Ranking model will use training data to learn this function.

Here we formulate a scoring function using a feed forward network. The function takes the features of a single example (i.e., query-document pair) and produces a relevance score

In [305]:
def example_feature_columns():
    """Returns the example feature columns."""
    feature_names = [
      "%d" % (i + 1) for i in range(0, _NUM_FEATURES)
    ]
    return {
      name: tf.feature_column.numeric_column(
          name, shape=(1,), default_value=0.0) for name in feature_names
    }

def make_score_fn(mode):
        """Returns a scoring function to build `EstimatorSpec`."""
        
        def _score_fn(context_features, group_features, mode, params, config):
            """Defines the network to score a documents."""
            del params
            del config
            # Define input layer.
            example_input = [
                tf.layers.flatten(group_features[name])
                for name in sorted(example_feature_columns())
            ]
            print("MAKE SCORE FUNCTION:")
            print(example_input)
            input_layer = tf.concat(example_input, 1)

            cur_layer = input_layer
            for i, layer_width in enumerate(int(d) for d in _HIDDEN_LAYER_DIMS):
                cur_layer = tf.layers.dense(
                  cur_layer,
                  units=layer_width,
                  activation="tanh")

            logits = tf.layers.dense(cur_layer, units=1)
            return logits
        return _score_fn

In [306]:
def eval_metric_fns():
    """Returns a dict from name to metric functions.

    This can be customized as follows. Care must be taken when handling padded
    lists.

    def _auc(labels, predictions, features):
    is_label_valid = tf_reshape(tf.greater_equal(labels, 0.), [-1, 1])
    clean_labels = tf.boolean_mask(tf.reshape(labels, [-1, 1], is_label_valid)
    clean_pred = tf.boolean_maks(tf.reshape(predictions, [-1, 1], is_label_valid)
    return tf.metrics.auc(clean_labels, tf.sigmoid(clean_pred), ...)
    metric_fns["auc"] = _auc

    Returns:
    A dict mapping from metric name to a metric function with above signature.
    """
    metric_fns = {}
    metric_fns.update({
      "metric/ndcg@%d" % topn: tfr.metrics.make_ranking_metric_fn(
          tfr.metrics.RankingMetricKey.NDCG, topn=topn)
      for topn in [1, 3, 5, 10]
    })

    return metric_fns

In [307]:
def get_estimator(hparams, mode):
    """Create a ranking estimator.

    Args:
    hparams: (tf.contrib.training.HParams) a hyperparameters object.

    Returns:
    tf.learn `Estimator`.
    """
    def _train_op_fn(loss):
        """Defines train op used in ranking head."""
        return tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.train.get_global_step(),
            learning_rate=hparams.learning_rate,
            optimizer="Adagrad")

    ranking_head = tfr.head.create_ranking_head(
      loss_fn=tfr.losses.make_loss_fn(_LOSS),
      eval_metric_fns=eval_metric_fns(),
      train_op_fn=_train_op_fn)

    return tf.estimator.Estimator(
      model_fn=tfr.model.make_groupwise_ranking_fn(
          group_score_fn=make_score_fn(mode),
          group_size=1,
          transform_fn=None,
          ranking_head=ranking_head),
        params=hparams)

In [308]:
hparams = tf.contrib.training.HParams(learning_rate=0.05)
ranker = get_estimator(hparams, tf.estimator.ModeKeys.TRAIN)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpvvsu96z2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f67fd655048>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [309]:
ranker.train(input_fn=lambda: input_fn(_TRAIN_DATA_PATH), steps=100)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
MAKE SCORE FUNCTION:
[<tf.Tensor 'groupwise_dnn_v2/group_score/flatten/Reshape:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'groupwise_dnn_v2/group_score/flatten_1/Reshape:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'groupwise_dnn_v2/group_score/flatten_2/Reshape:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'groupwise_dnn_v2/group_score/flatten_3/Reshape:0' shape=(?, 1) dtype=float32>]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.


/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpvvsu96z2/model.ckpt.
INFO:tensorflow:loss = 0.75400585, step = 1
INFO:tensorflow:Saving checkpoints for 100 into /tmp/tmpvvsu96z2/model.ckpt.
INFO:tensorflow:Loss for final step: 0.6130944.


In [310]:
ranker

In [290]:
def ltr_to_submission(df, ranker, path):
    import numpy as np
    preds = ranker.predict(input_fn=lambda: input_fn(path))
    import itertools
    preds_slice = itertools.islice(preds, _LIST_SIZE) # grab 
    count=0
    a = np.zeros((len(df), _LIST_SIZE))

    for i in preds_slice:
        a[count]=i
        count+=1
        
        
    test_X = df[[
       'sid',
        'transport_mode',
        'distance_plan',
        'eta', 
        'price'
    ]]
    
    # Assign prediction vals to df
    test_X = test_X.assign(yhat = a[:,0])
    
    df_end = pd.DataFrame(columns=['yhat'], index=df.sid.unique())

    df_end = test_X.sort_values(['sid', 'yhat'], ascending=False).groupby('sid').first()[[
        'yhat', 'transport_mode'
    ]]
    
    return df_end

In [317]:
df_train_train = pd.read_pickle("../data/interim/df_train_test.pickle")

In [312]:
df_train_test = pd.read_pickle("../data/interim/df_train_test.pickle")

In [319]:
df_preds = ltr_to_submission(df_train_train, ranker, '../data/interim/df_train_train.libsvm')

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
MAKE SCORE FUNCTION:
[<tf.Tensor 'groupwise_dnn_v2/group_score/flatten/Reshape:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'groupwise_dnn_v2/group_score/flatten_1/Reshape:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'groupwise_dnn_v2/group_score/flatten_2/Reshape:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'groupwise_dnn_v2/group_score/flatten_3/Reshape:0' shape=(?, 1) dtype=float32>]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpvvsu96z2/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [321]:
f1_score(df_train_train.groupby("sid").first()['click_mode'], df_preds.transport_mode, average='weighted')

0.3252145410735725

In [241]:
df_end = test_X[msk][['sid', 'transport_mode']]

In [265]:
test_X.groupby('sid', sort=False)['yhat'].transform(max)

143113     0.509255
143110     0.509255
143112     0.509255
143109     0.509255
459059     0.509255
459057     0.509255
459061     0.509255
459062     0.509255
459058     0.509255
459060     0.509255
909545     0.509255
909546     0.509255
909548     0.509255
909547     0.509255
241776    -1.318469
241774    -1.318469
241775    -1.318469
241773    -1.318469
241777    -1.318469
1270163    0.509255
1270164    0.509255
1270165    0.509255
1270168    0.509255
1270167    0.509255
1270166    0.509255
316955     0.509255
316959     0.509255
316957     0.509255
316958     0.509255
316956     0.509255
             ...   
1525598    0.000000
1525603    0.000000
1525602    0.000000
1525599    0.000000
1525600    0.000000
45680      0.000000
45682      0.000000
45681      0.000000
45678      0.000000
45679      0.000000
45683      0.000000
1449538    0.000000
1449537    0.000000
1449539    0.000000
1449535    0.000000
1449536    0.000000
1326911    0.000000
1326910    0.000000
1326912    0.000000


In [277]:
test_X['new']=test_X.groupby('sid')['yhat'].transform(max)

In [289]:
test_X.sort_values(['sid', 'yhat'], ascending=False).groupby('sid').first()

(129542, 6)

In [242]:
df_end.shape

(600983, 2)